 # MongoDB Game Agent Workshop: Graph-like Data Traversal



 **주제:** LLM 없이 구현하는 간단한 게임 NPC 에이전트



 이 노트북에서는 다음 내용을 다룹니다:

 1. **Graph-like Data Model:** MongoDB에서 관계형 데이터 탐색

 2. **Simple Game Agent:** 룰 기반 NPC 상호작용 시스템

 3. **Data Traversal:** User → Quest → Response 연결

 4. **Template-based Response:** LLM 없이 동적 응답 생성



 ---

 **시나리오:** World of Warcraft NPC (Thrall)가 플레이어의 질문에 답변



 **핵심 개념:**

 - 플레이어 데이터와 퀘스트 데이터를 연결하여 맥락있는 응답 생성

 - Graph traversal 패턴을 MongoDB 쿼리로 구현

 ## 환경 설정 및 데이터베이스 연결



 MongoDB에 연결하고 World of Warcraft 게임 데이터베이스를 초기화합니다.

In [ ]:
from pymongo import MongoClient

MONGODB_URI = "여기에_MONGODB_URI_입력"

# MongoDB 클라이언트 초기화
client = MongoClient(MONGODB_URI)
db = client["wow_workshop_db"]
collection = db["game_data"]

print("✅ Connected to World of Azeroth.\n")


 ## GameAgent 클래스 정의



 게임 에이전트는 다음과 같은 기능을 수행합니다:



 1. **플레이어 데이터 조회** (User Node)

 2. **퀘스트 데이터 조회** (Quest Node)

 3. **쿼리 처리 및 의도 분석**

 4. **템플릿 기반 응답 생성**



 ### Graph Structure:

 ```

 User → Active Quests → Quest Details → Response

 ```

In [ ]:
class GameAgent:
    """
    간단한 룰 기반 게임 NPC 에이전트
    MongoDB의 문서들을 그래프처럼 연결하여 탐색
    """
    
    def __init__(self, db_collection):
        """
        에이전트 초기화
        
        Args:
            db_collection: MongoDB collection 객체
        """
        self.collection = db_collection
    
    def get_player_data(self, user_id):
        """
        Node 1: User Data 조회
        플레이어의 캐릭터 정보와 현재 진행중인 퀘스트를 가져옴
        
        Args:
            user_id: 플레이어 고유 ID
            
        Returns:
            플레이어 데이터 문서
        """
        return self.collection.find_one({"type": "user_save", "user_id": user_id})
    
    def get_quest_data(self, quest_id):
        """
        Node 2: Quest Data 조회
        특정 퀘스트의 상세 정보를 가져옴
        
        Args:
            quest_id: 퀘스트 고유 ID
            
        Returns:
            퀘스트 데이터 문서
        """
        return self.collection.find_one({"type": "quest", "doc_id": quest_id})


 ## 쿼리 처리 및 의도 분석



 간단한 키워드 매칭을 통해 플레이어의 의도를 파악하고,

 적절한 핸들러로 라우팅합니다.

In [ ]:
    def process_query(self, user_id, query):
        """
        플레이어의 질문을 처리하고 적절한 응답 생성
        
        Args:
            user_id: 플레이어 ID
            query: 플레이어의 질문
            
        Returns:
            NPC의 응답 문자열
        """
        print(f"🤔 Processing query: '{query}'")
        print(f"   User ID: {user_id}")
        
        # 1. Identify Player (Graph Start Node)
        player = self.get_player_data(user_id)
        if not player:
            return "❌ Error: Player not found in database."
        
        print(f"   Found Player: {player['character_name']} (Lv.{player['level']} {player['faction']})")
        
        # 2. Intent Analysis (Simple Keyword Matching)
        # 실제로는 더 복잡한 NLP나 LLM을 사용할 수 있지만,
        # 여기서는 간단한 키워드 매칭 사용
        query_lower = query.lower()
        
        if "what" in query_lower and "do" in query_lower:
            return self.handle_quest_guidance(player)
        elif "who" in query_lower:
            return self.handle_identity(player)
        else:
            return "I don't understand. Ask me 'What should I do?'"


 ## 퀘스트 가이드 핸들러



 플레이어의 활성 퀘스트를 조회하고,

 해당 퀘스트의 상세 정보를 가져와 NPC 스타일의 응답을 생성합니다.

In [ ]:
    def handle_quest_guidance(self, player):
        """
        플레이어의 현재 퀘스트에 대한 가이드 제공
        
        Graph Traversal Path:
        User → active_quests → Quest Document → Response Template
        
        Args:
            player: 플레이어 데이터 문서
            
        Returns:
            퀘스트 가이드 응답
        """
        # 3. Graph Traversal: User -> Active Quests
        active_quests = player.get('active_quests', [])
        print(f"   Active Quests: {active_quests}")
        
        if not active_quests:
            return f"Champion {player['character_name']}, you have no active quests. Go rest!"
        
        # 4. Graph Traversal: Quest ID -> Quest Details
        current_quest_id = active_quests[0]
        quest_data = self.get_quest_data(current_quest_id)
        
        if not quest_data:
            return "Error: Quest data missing."
            
        # 5. Generate Response (Template Filling)
        # LLM 없이 파이썬 문자열 포맷팅으로 동적 응답 생성
        
        response = f"""
        ===========================================
        🛡️  WARCHIEF'S GUIDANCE (NPC: Thrall)
        ===========================================
        Lok'tar Ogar, {player['character_name']}!
        
        I see you are currently on the mission: 
        >> "{quest_data['title']}"
        
        📜 Mission Brief:
        {quest_data['content']}
        
        📍 Location: {quest_data['zone']}
        🎁 Reward: Honor and experience.
        
        *Go forth and bring glory to the {player['faction']}!*
        ===========================================
        """
        return response


 ## 플레이어 정체성 핸들러



 플레이어가 자신이 누구인지 물어볼 때의 응답을 처리합니다.

In [ ]:
    def handle_identity(self, player):
        """
        플레이어의 정체성에 대한 질문 처리
        
        Args:
            player: 플레이어 데이터 문서
            
        Returns:
            플레이어 정보 응답
        """
        return f"You are {player['character_name']}, a Level {player['level']} {player['class']} of the {player['faction']}."

# Add the methods to GameAgent class
GameAgent.process_query = process_query
GameAgent.handle_quest_guidance = handle_quest_guidance  
GameAgent.handle_identity = handle_identity


 ## 에이전트 초기화



 GameAgent 인스턴스를 생성하여 NPC 상호작용을 시작합니다.

In [ ]:
# Agent 초기화
agent = GameAgent(collection)
print("🤖 Game Agent initialized and ready!")


 ## Scenario 1: Horde 플레이어 상호작용



 오크 전사가 Warchief Thrall에게 길을 묻는 시나리오입니다.



 **예상 플로우:**

 1. 플레이어 데이터 조회 (user_orc_warrior)

 2. 활성 퀘스트 확인

 3. 퀘스트 상세 정보 조회

 4. Thrall의 응답 생성

In [ ]:
print("\n>>> [Scenario 1] Orc Warrior Interaction")
print("=" * 50)

user_id = "user_orc_warrior"
user_query = "Warchief, I am lost. What should I do?"

# 에이전트가 쿼리 처리
response = agent.process_query(user_id, user_query)
print(response)


 ## Scenario 2: Alliance 플레이어 상호작용



 인간 마법사가 퀘스트 가이드를 요청하는 시나리오입니다.



 **참고:** 실제 구현에서는 진영(Faction) 체크를 추가하여

 Horde NPC가 Alliance 플레이어에게 다르게 응답하도록 할 수 있습니다.

In [ ]:
print("\n>>> [Scenario 2] Human Mage Interaction")
print("=" * 50)

user_id_2 = "user_human_mage"
user_query_2 = "What should I do?"

# 에이전트가 쿼리 처리
# 현재 로직은 퀘스트를 찾아주지만, 
# 실제로는 진영 체크를 통해 다른 응답을 생성할 수 있음
response_2 = agent.process_query(user_id_2, user_query_2)
print(response_2)


 ## 추가 테스트: 플레이어 정체성 확인

In [ ]:
print("\n>>> [Additional Test] Identity Check")
print("=" * 50)

# 플레이어가 자신의 정체성을 물어보는 경우
identity_query = "Who am I?"
response_identity = agent.process_query("user_orc_warrior", identity_query)
print(response_identity)


 ## 결론 및 발전 방향



 ### 현재 구현:

 - **Simple Rule-based System**: 키워드 매칭으로 의도 파악

 - **Graph-like Traversal**: MongoDB 문서 간 관계 탐색

 - **Template-based Response**: 정적 템플릿으로 응답 생성



 ### 발전 가능한 방향:



 1. **LLM Integration**:

    - 더 자연스러운 대화 생성

    - 복잡한 의도 파악

    - 동적 스토리텔링



 2. **Advanced Graph Logic**:

    - 다중 관계 탐색 (Quest → NPC → Item)

    - 복잡한 조건부 로직 (레벨, 진영, 평판 체크)

    - 순환 그래프 탐색



 3. **State Management**:

    - 대화 히스토리 관리

    - 플레이어 선택 추적

    - 퀘스트 진행 상태 업데이트



 4. **Multi-Agent System**:

    - 여러 NPC 캐릭터

    - NPC 간 상호작용

    - 파티 시스템 지원

In [ ]:
print("\n✅ Workshop Complete!")
print("🎮 For the Horde! (또는 Alliance...)")